In [10]:
from time import sleep
import pydbus
      
import serial
import time
import re
import pandas as pd
import numpy as np
import pickle
import json

from IPython.display import IFrame
import serial.tools.list_ports

import pickle
import numpy as np
from pyargus.directionEstimation import *

import math
from collections import deque
from scipy.spatial import distance

from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.ticker import MultipleLocator

import pyaudio
import numpy as np
import matplotlib.pyplot as plt
import time
import wave
import IPython.display as ipd
import librosa.display
import sklearn
from scipy.stats import kurtosis, skew
import os

import csv
from playsound import playsound

import warnings
warnings.filterwarnings('ignore')

%run audio.ipynb
%run featurizer.ipynb
%run location.ipynb
%run utils.ipynb
%run direction.ipynb
%run WakeUp.ipynb

In [ ]:
path = 'data/feature.csv'
runtime = 5 #sec

f = Featurizer()
f.set_utils(location_id='Arena', user_id='user_01')

w = WakeUp()

idx = 0
while True:
    if w.wake_up():
        playsound('util/yes.wav')

        audio = f.get_audio_features(idx, runtime) 
        metadata = f.get_metadata()
        coord=  f.get_location()
        direction=  f.get_direction()
        rf = f.get_rf_attributes()
        
        row = pd.concat([metadata, coord, direction, audio, rf], axis=1)

        df = f.load_data(path)
        df = pd.concat([df, row], axis=0)
        df.to_csv(path, index=False)

        print("\n\n----------------------------")
        
        idx +=1
        
        playsound('util/okay.wav')
    
f.close_connections()